# <center>Facial Emotion Recognition</center>

## Introduction
Facial emotion recognition using Convolutional Neural Networks (CNN) is a technique to  recognize human emotions from facial expressions in images.<br>
The objective of this project is to categorize each face based on the emotion shown in the facial expression into one of seven categories (0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral). 
### Approach
<li>A CNN model is trained on the FER dataset to learn the features of facial expressions that are associated with each emotion. 
<li>The model uses convolutional layers to extract features from the input images and pooling layers to reduce the dimensionality of the feature maps. 
<li>The extracted features are then fed into fully connected layers to classify the emotion.

## Dataset Description
<li>The dataset consists of a collection of grayscale images (48x48 pixel) of human faces, where each image is labeled with one of seven basic emotions: 
anger, disgust, fear, happiness, sadness, surprise, or neutral.<br>
<li>The training set consists of 28,709 examples.<br>
<li>The test set consists of over 7,000 samples

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping


## Loading the Dataset

In [ ]:
train_dir = "/kaggle/input/fer2013/train"  # train data folder
test_dir = "/kaggle/input/fer2013/test"    # test data folder

#### Create a function to do the following
<li> It takes in the folder name as input and loads all the images in that folder along with their corresponding labels. 
<li>It then loops through all the subfolders (labels) in the given folder and then loops through all the files (images) in each subfolder.
<li>For each image file, it constructs the full path of the image using and appends it to the `image_paths` list. The corresponding label is also appended to the `labels` list.
<li>Finally, the function returns the two lists.

In [ ]:
def load_dataset(directory):
    image_paths = []
    labels = []
    
    for label in os.listdir(directory):
      
        for filename in os.listdir(directory+"/"+label):
            image_path = os.path.join(directory, label, filename)
            image_paths.append(image_path)
            labels.append(label)

    return image_paths, labels

In [ ]:
## convert into dataframe
train = pd.DataFrame()
train['image'], train['label'] = load_dataset(train_dir)
train.head()

In [ ]:
train.tail()

In [ ]:
train.shape

In [ ]:
train['label'].value_counts()

#### Similarly, construct a dataframe for Test data

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = load_dataset(test_dir)
test.head()

In [ ]:
test.shape

In [ ]:
test['label'].value_counts()

##### Let's load an image file using the <b>Python Imaging Library (Pillow) </b>and display it using Matplotlib.

In [ ]:
from PIL import Image
# Load image using Pillow
img = Image.open(train['image'][0])
plt.imshow(img, cmap='gray');

In [ ]:
img

#### Check the Image size
<li> `size` attribute returns a tuple of the width and height of the image in pixels.
<li><b>'img.getbands()'</b> method returns a tuple of the band names in the image. <br>
    For example, an RGB image has three bands: red, green, and blue.<br>
    If the image is grayscale, getbands method of the image object will return a tuple containing a single string element "L". "L" stands for "Luminance", which is the intensity of the pixel's brightness.



In [ ]:
shape = img.size + img.getbands() 
print("The shape of the image is:", shape)

In [ ]:
def resizing(images):
    features = []
    for image in tqdm(images):
        img = Image.open(image)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

<li>The above function takes a list of image paths as input, opens each image using the PIL Image module, converts it into a numpy array and appends it to a list called features. 
<li>The `tqdm()` function is used to display a progress bar during the loop execution.
<li>After processing all images in the input list, the features list is converted into a numpy array and reshaped into a four-dimensional array with dimensions <b>(number of samples, height, width, channels)</b>, 
<li>Each image is resized to a square image with dimensions 48x48 and a single color channel, which is suitable for inputting into a neural network.   

In [ ]:
train_features = resizing(train['image'])

In [ ]:
train_features.shape

In [ ]:
test_features = resizing(test['image'])

In [ ]:
test_features.shape

In [ ]:
## normalize the image
x_train = train_features/255
x_test = test_features/255

In [ ]:
## Encoding the output column (label)

from sklearn.preprocessing import LabelEncoder
la = LabelEncoder()
la.fit(train['label'])
y_train = la.transform(train['label'])
y_test = la.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [ ]:
y_train[0], y_train[-1]

## Model Creation

In [ ]:
input_shape = (48, 48, 1)
output_class = 7

In [ ]:
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
# train the model
history = model.fit(x = x_train, y = y_train, batch_size = 32, epochs = 100, validation_data=(x_test, y_test))

In [ ]:
history = model.fit(x = x_train, y = y_train, batch_size = 32, epochs = 20, validation_data=(x_test, y_test),callbacks=[early_stop])

## Predict with test images

In [ ]:
image_index = random.randint(0, len(test))
print("Original Output:", test['label'][image_index])
pred = model.predict(x_test[image_index].reshape(1, 48, 48, 1))
prediction_label = la.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.imshow(x_test[image_index], cmap='gray');

In [ ]:
image_index = random.randint(0, len(test))
print("Original Output:", test['label'][image_index])
pred = model.predict(x_test[image_index].reshape(1, 48, 48, 1))
prediction_label = la.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.subplots(figsize=(1, 1))
plt.imshow(x_test[image_index], cmap='gray');


In [ ]:
image_index = random.randint(0, len(test))
print("Original Output:", test['label'][image_index])
pred = model.predict(x_test[image_index].reshape(1, 48, 48, 1))
prediction_label = la.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.subplots(figsize=(1,1))
plt.imshow(x_test[image_index], cmap='gray');

In [ ]:
image_index = random.randint(0, len(test))
print("Original Output:", test['label'][image_index])
pred = model.predict(x_test[image_index].reshape(1, 48, 48, 1))
prediction_label = la.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label)
plt.subplots(figsize=(1,1))
plt.imshow(x_test[image_index], cmap='gray');